In [4]:
import zstandard
import io
import json
import time
import threading

from tqdm import tqdm

filepath = "D:\\Torrents\\Subreddits\\reddit\\subreddits24\\investing_discussion_submissions.zst"

def process_zst_file(filepath, stop_flag):
    with open(filepath, 'rb') as compressed_file:
        dctx = zstandard.ZstdDecompressor()
        with dctx.stream_reader(compressed_file) as reader:
            # Use io.TextIOWrapper to treat the decompressed stream as text
            text_reader = io.TextIOWrapper(reader, encoding='utf-8')  # Adjust encoding if needed

            i = 0  # Initialize count
            out_list = []
            unique_subreddits = set()
            for line in tqdm(text_reader):
                if stop_flag.is_set():
                    print("Processing interrupted!")
                    return unique_subreddits, out_list  # Return partial results

                try:
                    data = json.loads(line)  # Parse the JSON string into a dictionary
                    subreddit = data["subreddit"]
                    if subreddit == "WallStreetBets":
                        print("here")
                        out_list.append(data)
                    else:
                        unique_subreddits.add(subreddit)
                        i += 1
                        out_list.append(data)
                        # No need for the fixed count break here

                except json.JSONDecodeError as e:
                    print(f"Error decoding JSON on line: {line.strip()}. Error: {e}")
                    # Handle the error, maybe skip the line or log it
                    continue
            print("Finished processing the entire file.")
            return unique_subreddits, out_list



In [5]:
stop_event = threading.Event()
results = None

def run_processor():
    global results
    results = process_zst_file(filepath, stop_event)

processor_thread = threading.Thread(target=run_processor)
processor_thread.start()


0it [00:00, ?it/s]

26923it [00:01, 15733.94it/s]


Finished processing the entire file.


In [ ]:

# Simulate some time passing, then decide to interrupt
print("Sending interrupt signal...")
stop_event.set()
processor_thread.join()  # Wait for the thread to finish (gracefully)

if results:
    unique_subs, data_list = results
    print("\\nResults after interruption:")
    print(f"Number of unique subreddits found: {len(unique_subs)}")
    print(f"Number of data entries collected: {len(data_list)}")
else:
    print("\\nNo results obtained (function might have been interrupted very early).")

Sending interrupt signal...

Results after interruption:
Number of unique subreddits found: 6823
Number of data entries collected: 24628


In [7]:
results[1][-1]

{'_meta': {'removal_type': 'reddit',
  'retrieved_2nd_on': 1735812366,
  'was_deleted_later': True},
 'all_awardings': [],
 'allow_live_comments': False,
 'approved_at_utc': None,
 'approved_by': None,
 'archived': False,
 'author': 'Dripbydaylight',
 'author_flair_background_color': None,
 'author_flair_css_class': None,
 'author_flair_richtext': [],
 'author_flair_template_id': None,
 'author_flair_text': None,
 'author_flair_text_color': None,
 'author_flair_type': 'text',
 'author_fullname': 't2_dfluy9hv',
 'author_is_blocked': False,
 'author_patreon_flair': False,
 'author_premium': False,
 'awarders': [],
 'banned_at_utc': None,
 'banned_by': None,
 'can_gild': False,
 'can_mod_post': False,
 'category': None,
 'clicked': False,
 'content_categories': None,
 'contest_mode': False,
 'created': 1735682751.0,
 'created_utc': 1735682751.0,
 'discussion_type': None,
 'distinguished': None,
 'domain': 'self.investing_discussion',
 'downs': 0,
 'edited': False,
 'gilded': 0,
 'gildings

In [20]:
import datetime
import pytz

utc_timestamp_seconds = 1546373162
utc_timezone = pytz.utc
utc_aware_datetime = datetime.datetime.fromtimestamp(utc_timestamp_seconds, tz=utc_timezone)

# Format the datetime object
formatted_date = utc_aware_datetime.strftime("%Y-%m-%d %H:%M:%S %Z%z")
print(f"Formatted UTC date: {formatted_date}")


Formatted UTC date: 2019-01-01 20:06:02 UTC+0000
